In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification, pipeline
import pandas as pd
import numpy as np

# Carga Datos

In [ ]:
train200k = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/train200k.csv", sep=',')

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train200k, test_size =0.1)

In [ ]:
val['humor'].value_counts()

# Me quedo con 1400 opiniones positivas
np.random.seed(1)
# get_n = 1400
remove_n = 4000 # data.query('Sentiment == 2').shape[0] - get_n
drop_indices = np.random.choice(val.query('humor == False').index, remove_n, replace=False)
text = val.drop(drop_indices)

text['humor'].value_counts()

True     7965
False    4035
Name: humor, dtype: int64

In [ ]:
text

,text,humor
151012,Sheriff where sandra bland died says there's n...,False
77521,Healthy indian takeout: 8 tips for making smar...,False
90117,I really hope that death is a woman. that way ...,True
135494,My therapist said that i second guess myself t...,True
17293,What do you call a math teacher who's really i...,True
...,...,...
5056,How come sneezes get a god bless you but cough...,True
150502,Nick kristof: it's 'bogus' to say hillary is f...,False
53854,"In foreclosure-ridden florida, 'zombie' swimmi...",False
22264,What do you call a man who can't stand? neal,True


---
---

# Word Emoji Context Matrix. 🤣 😆 😂 

Descripción: Extracción/guardado de Emojis y depuración del Corpus

Inconvenientes detectados con los emojis:

* 1 Resulta que los emojis hemos podido verificar que countvectorizer los ignora. Es decir si las frases fueran solo emojis el shape de la matriz word context sería (1,1), con valor empty.

* 2 Aun suponiendo que exista alguna alternativa a countvectorizer muchas personas escriben palabras juntas con emojis, e.g.: 'man👨', lo que podría generar un token más del vocabulario que innecesario. Además pueden escribir varios emojis juntos 👨, 👨👨, 👨👨👨, ... generando también un problema en su identificación.

La propuesta ha sido realizar un preproceso como el siguiente. Además, para solucionar el problema de los emoticonos, en lugar de pasárselos a countvectorizer en bruto utilizamos la decodificación a texto de la librería emoji: **emoji.demojize()**

In [ ]:
# %%capture
!pip install emoji

In [ ]:
import emoji

In [ ]:
def extract_emojis(tweet):
  return ''.join(c for c in tweet if c in emoji.UNICODE_EMOJI_ENGLISH)

def get_emojilist(tweet):
  emojistring = extract_emojis(tweet)
  emojilist = [emoji for emoji in emojistring]
  for emoji in emojilist:
    tweet = tweet.replace(emoji, "")
  return tweet, emojilist

def get_emoji_sentence(tweet):
  tweet, emojilist = get_emojilist(tweet)
  emoji_sentence = emoji.demojize(' '.join(emojilist))
  return tweet, emoji_sentence

def filter(text_df):
  filtered_df = pd.DataFrame(map(get_emoji_sentence, text_df['text'].to_list()))
  filtered_df.rename(columns={0: 'sentences_without_emojis', 1: 'emoji_sentences'}, inplace=True)
  return filtered_df

df_filter_output = filter(text)

df_cleaned_sentences = df_filter_output[['sentences_without_emojis']]
# df_cleaned_sentences = df_filter_output['emoji_sentences']

text['text_v1'] = df_filter_output['sentences_without_emojis'].to_list()

emoji_sentences = df_filter_output['emoji_sentences']

text['text_emojis'] = emoji_sentences.to_list()


def get_emoji_vocab(emoji_sentence):
  analyzer = CountVectorizer(binary=False, analyzer='word', # stop_words=more_stop_words,
                              ngram_range=(1, 1)).build_analyzer()
  return (emoji for emoji in analyzer(emoji_sentence))

In [ ]:
from sklearn.feature_extraction.text import  CountVectorizer
cv_emoji = CountVectorizer(analyzer=get_emoji_vocab)

In [ ]:
try:
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
# Just if there is no emoji
except ValueError:
  emoji_sentences.loc[0,0] = emoji.demojize("🆘")
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
  """emoji_sentences_test = emoji_sentences"""
  """emoji_sentences_test.loc[0,0] = "🆘🆘🆘🆘🆘🆘🆘🆘"""
  """text['text_emojis'] = emoji_sentences_test.to_list()"""
# cv_emoji.get_feature_names()


text.replace({'text_emojis': {'': 'no_emojis'}}, inplace= True)

corpus = text.copy()

In [ ]:
cv_emoji.get_feature_names()

['sos_button']

In [ ]:
text

,text,humor,text_v1,text_emojis
151012,Sheriff where sandra bland died says there's n...,False,Sheriff where sandra bland died says there's n...,no_emojis
77521,Healthy indian takeout: 8 tips for making smar...,False,Healthy indian takeout: 8 tips for making smar...,no_emojis
90117,I really hope that death is a woman. that way ...,True,I really hope that death is a woman. that way ...,no_emojis
135494,My therapist said that i second guess myself t...,True,My therapist said that i second guess myself t...,no_emojis
17293,What do you call a math teacher who's really i...,True,What do you call a math teacher who's really i...,no_emojis
...,...,...,...,...
5056,How come sneezes get a god bless you but cough...,True,How come sneezes get a god bless you but cough...,no_emojis
150502,Nick kristof: it's 'bogus' to say hillary is f...,False,Nick kristof: it's 'bogus' to say hillary is f...,no_emojis
53854,"In foreclosure-ridden florida, 'zombie' swimmi...",False,"In foreclosure-ridden florida, 'zombie' swimmi...",no_emojis
22264,What do you call a man who can't stand? neal,True,What do you call a man who can't stand? neal,no_emojis


# Detector de cansinos y corrección de repeticiones innecesarias

En twitter hay mucho "pesao" con el pulgar "cansao" y lo dejan mucho tiempo en el móooooooooovil pulsando una letraaaaa. A todos estos, los consideramos unos pesaos y lo vamos a tener en cuenta. No quieren trabajar.

¿Por qué corregiremos también a los pesaos? Porque spellchecker no puede, es fácil ver esto con un ejemplo en código. 

In [ ]:
def is_cansino(tweet, tolerancia:int=2):
  # Consideramos pesados a todos aquellos
  # que repitan 3 veces una letra
  """
  Esta función identifica a los pesaos
  Args:
    tolerancia (int): tolerancia al pesao
  """
  l1 = ""
  rep = 0
  letra = ""
  tweet_aux = tweet + '<eos>'
  for index, letra in enumerate(tweet_aux):
      if letra == l1:
          rep += 1
      elif rep >= tolerancia:
          return l1, rep, tolerancia, 1*True
      else:
        rep = 0
        l1 = letra
  return letra, 0, tolerancia, 1*False

def get_cansinos(corpus):
  L = list(map(is_cansino, corpus))
  return [ret[3] for ret in L]

###################################Ç
##### Correccion repeticiones #####
###################################

# Esto es beta solo corrige la primera
# tanda de repeticiones se puede mejorar

def tweet_corrector(tweet):
    letra, rep, tolerancia, bool_ = is_cansino(tweet)
    if bool_ == 1:
        tweet = tweet.replace(rep*letra, '')
        tweet = tweet_corrector(tweet)
    return tweet

def corpus_corrector(corpus):
    L = list(map(tweet_corrector, corpus))
    return L

In [ ]:
"""
text.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'
text
"""

"\ntext.loc[7,'text_v2'] = 'Meeee aaaaabuuuuurroooooo'\ntext\n"

In [ ]:
text['detected_text_cansinos'] = get_cansinos(text['text_v1'].to_list())
text['detected_emojis_cansinos'] = get_cansinos(text['text_emojis'].to_list())
text['detected_cansinos'] = (text['detected_text_cansinos'] + text['detected_emojis_cansinos']) % 2
# text['detected_emojis_cansinos] | text['detected_text_cansinos'] 

text['text_v2'] = corpus_corrector(text['text_v1'].to_list())

text

,text,humor,text_v1,text_emojis,detected_text_cansinos,detected_emojis_cansinos,detected_cansinos,text_v2
151012,Sheriff where sandra bland died says there's n...,False,Sheriff where sandra bland died says there's n...,no_emojis,0,0,0,Sheriff where sandra bland died says there's n...
77521,Healthy indian takeout: 8 tips for making smar...,False,Healthy indian takeout: 8 tips for making smar...,no_emojis,0,0,0,Healthy indian takeout: 8 tips for making smar...
90117,I really hope that death is a woman. that way ...,True,I really hope that death is a woman. that way ...,no_emojis,0,0,0,I really hope that death is a woman. that way ...
135494,My therapist said that i second guess myself t...,True,My therapist said that i second guess myself t...,no_emojis,0,0,0,My therapist said that i second guess myself t...
17293,What do you call a math teacher who's really i...,True,What do you call a math teacher who's really i...,no_emojis,0,0,0,What do you call a math teacher who's really i...
...,...,...,...,...,...,...,...,...
5056,How come sneezes get a god bless you but cough...,True,How come sneezes get a god bless you but cough...,no_emojis,0,0,0,How come sneezes get a god bless you but cough...
150502,Nick kristof: it's 'bogus' to say hillary is f...,False,Nick kristof: it's 'bogus' to say hillary is f...,no_emojis,0,0,0,Nick kristof: it's 'bogus' to say hillary is f...
53854,"In foreclosure-ridden florida, 'zombie' swimmi...",False,"In foreclosure-ridden florida, 'zombie' swimmi...",no_emojis,0,0,0,"In foreclosure-ridden florida, 'zombie' swimmi..."
22264,What do you call a man who can't stand? neal,True,What do you call a man who can't stand? neal,no_emojis,0,0,0,What do you call a man who can't stand? neal


---
---

# Traducción Corpus

# Descripción:

Frente a la adversidad que supone que la gran mayoría de modelos desarrollados en HuggingFace se encuentran en Inglés, se ha optado por traducir el corpus al inglés con el pipeline correspondiente. Así, podemos aprovecharlos para hacer Transfer Learning.

Puesto que la fama de los transformers les antecede, consideramos que el error de los mismos de cara a las traducciones resulta asumible.

In [ ]:
# Autotokenizador
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")

# Elección/Descarga del modelo Preentrenado
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

In [ ]:
def get_corpus_translation(corpus):
  from transformers import pipeline
  pline = pipeline("translation_en_to_es", model=model, tokenizer=tokenizer)

  def get_tweet_translation(tweet):

    tweet = tweet.lower()

    tweet = tweet.replace('.', ' <.> ')
    tweet = tweet.replace('...', ' . ')
    

    return pline(tweet)[0]['translation_text'] # punto <point>

  return list(map(get_tweet_translation, corpus['text_v2'].to_list()))

text['es_text'] = get_corpus_translation(text)

def recover_point(tweet):
  return tweet.replace('<.>', ' . ')

def recover_point_corpus(corpus):
  return list(map(recover_point, corpus))

text['es_text'] = recover_point_corpus(text['es_text'])



In [ ]:
text.to_csv('AumentadoDatos_EN.csv')